<a href="https://colab.research.google.com/github/FATCK06/ProjectAPI_FirstSemester/blob/BranchPai_BackEnd/RubyFox_Analise_Populacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extraindo dados populacional de São José dos Campos:**

Nosso Github: https://github.com/FATCK06/ProjectAPI_FirstSemester

Desenvolvemos esse código dentro do Google Colab para apresentar dados do censo de IBGE dos anos 2010 e 2022, tais informações apresenta: População residente de SJC, Faixa etária, Densidade demográfica e População por sexo.

# Instalando bibliotecas Python para o progama:

Base de todo programa para funcionar.

In [ ]:
!pip install flask flask-ngrok pyngrok matplotlib plotly pandas sidrapy streamlit flask-cors geopandas geobr --upgrade

# Inserindo Token Pyngrok:

Importante, não esquecer de rodar isso primeiro.

In [1]:
!ngrok config add-authtoken 32Qi64IHMghRwMNkoFZu49G4xIF_yYjaH7aJoXcfiJEhhhCs

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# Carregando dados CSV do IBGE:

Verificação via link público e analisar se os dados estão corretos.

In [ ]:
import pandas as pd
import requests

# ==============================================================================
# Cole aqui o dicionário de URLs do seu script do dashboard
# ==============================================================================
urls_csv = {
    "idade_sexo_2022": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20Idade%20e%20sexo.csv",
    "pop_residencia_2022": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20popula%C3%A7%C3%A3o%20e%20residencia.csv",
    "pop_residencia_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/populacao_residente_sjc_2010.csv",
    "densidade_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/densidade_demografica_sjc_2010.csv",
    "faixa_mulheres_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/faixa_etaria_mulheres_2010.csv",
    "faixa_homens_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/faixa_etaria_homens_2010.csv"
}
# ==============================================================================

print("─" * 60)
print("🚀 Iniciando a verificação de todos os links dos arquivos CSV...")
print("─" * 60)

links_com_falha = 0

# Itera sobre cada nome e url no dicionário
for nome_arquivo, url in urls_csv.items():
    print(f"Verificando '{nome_arquivo}'...")
    print(f"URL: {url}")

    try:
        # Tenta ler apenas as primeiras 5 linhas para ser mais rápido (head)
        # O Pandas precisa de um 'User-Agent' para evitar ser bloqueado (erro 403)
        # Este cabeçalho simula um navegador web.
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
        }
        # Primeiro, verificamos se o link existe (status 200 OK)
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Lança um erro se o status não for 200

        # Se o link estiver OK, tentamos ler com o Pandas
        df_teste = pd.read_csv(url)
        print("✅ SUCESSO: O link é válido e o arquivo CSV foi lido corretamente.")

    except requests.exceptions.HTTPError as http_err:
        print(f"❌ FALHA: Erro de HTTP. O link parece estar quebrado ou inacessível. (Status: {http_err.response.status_code})")
        links_com_falha += 1

    except pd.errors.ParserError as parser_err:
        print(f"❌ FALHA: O link é válido, mas o arquivo não parece ser um CSV bem formatado. (Erro de Parser)")
        links_com_falha += 1

    except Exception as e:
        print(f"❌ FALHA: Ocorreu um erro inesperado. (Erro: {e})")
        links_com_falha += 1

    print("-" * 50)

print("\n" + "─" * 60)
print("🏁 Verificação Concluída!")
if links_com_falha == 0:
    print("🎉 Ótima notícia! Todos os links foram verificados e estão funcionando perfeitamente.")
else:
    print(f"⚠️ Atenção: {links_com_falha} link(s) apresentaram falha. Revise as URLs com problema.")
print("─" * 60)


# Ainda sobre construção, será onde o programa rodará com Flask juntamente com streamlit:

Como dito antes, ainda está em construção então falta inserir mais algumas coisas, como o Flask.

In [ ]:
import os
import socket
import subprocess
import time
import atexit
from pyngrok import ngrok

# ==================================================
# 1. Código do Streamlit com carregamento via URL
# ==================================================
# Usamos r"""...""" para criar uma string de múltiplas linhas que será salva no arquivo.
streamlit_code = r"""
import streamlit as st
import pandas as pd
import plotly.express as px
import re

st.set_page_config(
    page_title="Dashboard Populacional SJC",
    page_icon="📊",
    layout="wide"
)

# =========================
# Funções auxiliares
# =========================

@st.cache_data
def limpar_colunas(df):
    cols = df.columns.str.strip().str.lower()
    replacements = {
        ' ': '_', 'ç': 'c', 'ã': 'a', 'õ': 'o', 'á': 'a', 'é': 'e',
        'í': 'i', 'ó': 'o', 'ú': 'u', 'â': 'a', 'ê': 'e', 'î': 'i',
        'ô': 'o', 'û': 'u'
    }
    for old, new in replacements.items():
        cols = cols.str.replace(old, new, regex=False)
    cols = cols.str.replace(r'[^a-z0-9_]', '', regex=True)
    df.columns = cols
    return df

def converter_para_numerico(series):
    if pd.api.types.is_string_dtype(series):
        series = (
            series.str.replace(r"(dados alterados)", "", regex=True)
            .str.replace(r"\\.", "", regex=True)
            .str.replace(r",", ".", regex=True)
        )
    return pd.to_numeric(series, errors='coerce')

@st.cache_data
def carregar_csvs():
    # ==============================================================================
    # <<< ÁREA DE ALTERAÇÃO DOS LINKS >>>
    # Altere os links abaixo para apontar para os seus arquivos CSV no GitHub.
    # Lembre-se de usar o link "Raw" (Bruto) de cada arquivo.
    # Dica: Evite usar caracteres especiais e acentos nos nomes dos arquivos
    #       para facilitar a criação das URLs.
    # ==============================================================================
    urls_csv = {
        "idade_sexo_2022": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20Idade%20e%20sexo.csv",
        "pop_residencia_2022": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20popula%C3%A7%C3%A3o%20e%20residencia.csv",
        "pop_residencia_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/populacao_residente_sjc_2010.csv",
        "densidade_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/densidade_demografica_sjc_2010.csv",
        "faixa_mulheres_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/faixa_etaria_mulheres_2010.csv",
        "faixa_homens_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/faixa_etaria_homens_2010.csv"
    }
    # ==============================================================================

    try:
        # Carrega os dados brutos a partir das URLs
        datasets_raw = {key: pd.read_csv(url) for key, url in urls_csv.items()}

        # Cria uma cópia limpa para manipulação
        datasets = {}
        for key, df in datasets_raw.items():
            # Preserva os dados originais de 2010 para busca de strings
            if key in ["pop_residencia_2010", "faixa_homens_2010", "faixa_mulheres_2010"]:
                datasets[key] = df.copy()
            # Limpa os nomes das colunas dos outros
            else:
                datasets[key] = limpar_colunas(df.copy())

        # Converte colunas numéricas dos arquivos de 2022
        for col in datasets["idade_sexo_2022"].columns:
            if col not in ["tipo", "ano"]:
                datasets["idade_sexo_2022"][col] = converter_para_numerico(datasets["idade_sexo_2022"][col])

        for col in datasets["pop_residencia_2022"].columns:
            if col not in ['ano']:
                datasets["pop_residencia_2022"][col] = converter_para_numerico(datasets["pop_residencia_2022"][col])

        return datasets

    except Exception as e:
        st.error(f"Ocorreu um erro ao carregar os dados de uma URL. Verifique os links no código. Erro: {e}")
        st.stop()

def normalizar_faixa_etaria_2010(indicador):
    indicador = str(indicador).lower()
    if 'menos de 1' in indicador or '1 a 4' in indicador: return '0_a_4_anos'
    if '5 a 9' in indicador: return '5_a_9_anos'
    if '10 a 14' in indicador: return '10_a_14_anos'
    if '15 a 19' in indicador: return '15_a_19_anos'
    if '20 a 24' in indicador: return '20_a_24_anos'
    if '25 a 29' in indicador: return '25_a_29_anos'
    if '30 a 34' in indicador: return '30_a_34_anos'
    if '35 a 39' in indicador: return '35_a_39_anos'
    if '40 a 44' in indicador: return '40_a_44_anos'
    if '45 a 49' in indicador: return '45_a_49_anos'
    if '50 a 54' in indicador: return '50_a_54_anos'
    if '55 a 59' in indicador: return '55_a_59_anos'
    if '60 a 64' in indicador: return '60_a_64_anos'
    if '65 a 69' in indicador: return '65_a_69_anos'
    if '70 a 74' in indicador: return '70_a_74_anos'
    if '75 a 79' in indicador: return '75_a_79_anos'
    if '80 a 84' in indicador: return '80_a_84_anos'
    if '85 a 89' in indicador: return '85_a_89_anos'
    if '90 a 94' in indicador: return '90_a_94_anos'
    if '95 a 99' in indicador: return '95_a_99_anos'
    if '100 anos ou mais' in indicador: return '100_anos_ou_mais'
    return None

# =========================
# Título Principal
# =========================
st.title("📊 Análise Populacional - São José dos Campos")
datasets = carregar_csvs()

# --- Extração de Totais ---
total_correto_2022 = datasets["pop_residencia_2022"]["populacao_residente"].iloc[0]
df_pop_2010 = datasets["pop_residencia_2010"]
# Busca pelo valor exato no CSV original, antes da limpeza de colunas
total_correto_2010 = df_pop_2010.loc[df_pop_2010['Indicador'] == 'População residente', 'Valor'].iloc[0]


# ===================================================================
# GRÁFICO 1: POPULAÇÃO POR SEXO & MÉTRICAS
# ===================================================================
st.header("População Geral")
col1, col2 = st.columns([3, 1])

with col1:
    # --- Cálculo 2022 ---
    df_idade_sexo_2022 = datasets["idade_sexo_2022"]
    homens_2022_raw = df_idade_sexo_2022[df_idade_sexo_2022["tipo"].str.contains("homens", na=False, case=False)]["total"].sum()
    mulheres_2022_raw = df_idade_sexo_2022[df_idade_sexo_2022["tipo"].str.contains("mulheres", na=False, case=False)]["total"].sum()
    soma_raw_2022 = homens_2022_raw + mulheres_2022_raw
    homens_2022 = (homens_2022_raw / soma_raw_2022) * total_correto_2022 if soma_raw_2022 > 0 else 0
    mulheres_2022 = (mulheres_2022_raw / soma_raw_2022) * total_correto_2022 if soma_raw_2022 > 0 else 0
    df_sexo_2022 = pd.DataFrame([{"Sexo": "Homens", "População": homens_2022, "Ano": "2022"}, {"Sexo": "Mulheres", "População": mulheres_2022, "Ano": "2022"}])

    # --- Cálculo 2010 ---
    homens_2010_raw = datasets["faixa_homens_2010"]['Valor'].sum()
    mulheres_2010_raw = datasets["faixa_mulheres_2010"]['Valor'].sum()
    soma_raw_2010 = homens_2010_raw + mulheres_2010_raw
    homens_2010 = (homens_2010_raw / soma_raw_2010) * total_correto_2010 if soma_raw_2010 > 0 else 0
    mulheres_2010 = (mulheres_2010_raw / soma_raw_2010) * total_correto_2010 if soma_raw_2010 > 0 else 0
    df_sexo_2010 = pd.DataFrame([{"Sexo": "Homens", "População": homens_2010, "Ano": "2010"}, {"Sexo": "Mulheres", "População": mulheres_2010, "Ano": "2010"}])

    df_sexo_final = pd.concat([df_sexo_2010, df_sexo_2022], ignore_index=True)
    fig1 = px.bar(df_sexo_final, x="Sexo", y="População", color="Ano", barmode="group", title="População por Sexo (2010 vs 2022)", text_auto='.3s')
    fig1.update_traces(textangle=0, textposition="outside")
    st.plotly_chart(fig1, use_container_width=True)

with col2:
    st.metric("População Total 2022", f"{int(total_correto_2022):,}".replace(",", "."))
    st.metric("População Total 2010", f"{int(total_correto_2010):,}".replace(",", "."))
    if total_correto_2010 > 0:
        st.metric("Crescimento Populacional", f"{(total_correto_2022/total_correto_2010 - 1):.2%}")

# ===================================================================
# GRÁFICO 2: DENSIDADE DEMOGRÁFICA
# ===================================================================
st.header("Densidade Demográfica")
# CORREÇÃO: Os nomes das colunas de densidade são diferentes, então precisamos tratar isso.
df_densidade_2010 = limpar_colunas(datasets["densidade_2010"].copy())

data_densidade = []
if "densidade_demografica_habkm2" in df_densidade_2010.columns:
    densidade_2010 = df_densidade_2010["densidade_demografica_habkm2"].iloc[0]
    data_densidade.append({"Ano": "2010", "Densidade (hab/km²)": densidade_2010})
if "densidade" in datasets["pop_residencia_2022"].columns:
    densidade_2022 = datasets["pop_residencia_2022"]["densidade"].iloc[0]
    data_densidade.append({"Ano": "2022", "Densidade (hab/km²)": densidade_2022})

if data_densidade:
    df_densidade = pd.DataFrame(data_densidade)
    fig2 = px.line(df_densidade, x="Ano", y="Densidade (hab/km²)", markers=True, title="Densidade Demográfica (2010 vs 2022)")
    fig2.update_traces(text=df_densidade["Densidade (hab/km²)"].round(2), textposition="top center")
    st.plotly_chart(fig2, use_container_width=True)
else:
    st.warning("Dados de densidade demográfica não disponíveis.")


# ===================================================================
# GRÁFICO 3: FAIXA ETÁRIA (2010 vs 2022)
# ===================================================================
st.header("Distribuição da População por Faixa Etária")
# --- Cálculo 2022 ---
faixas_2022 = [c for c in df_idade_sexo_2022.columns if c not in ["ano", "tipo", "total"]]
faixa_total_2022_raw = df_idade_sexo_2022.loc[df_idade_sexo_2022["tipo"].str.lower().isin(["homens", "mulheres"]), faixas_2022].sum().fillna(0)
fator_2022 = total_correto_2022 / faixa_total_2022_raw.sum() if faixa_total_2022_raw.sum() > 0 else 1
faixa_total_2022 = (faixa_total_2022_raw * fator_2022).reset_index(); faixa_total_2022.columns = ["faixa_etaria", "populacao"]; faixa_total_2022["ano"] = "2022"

# --- Cálculo 2010 ---
df_h_2010 = datasets["faixa_homens_2010"]; df_m_2010 = datasets["faixa_mulheres_2010"]
df_faixa_2010_combined = pd.concat([df_h_2010, df_m_2010])
df_faixa_2010_combined['faixa_normalizada'] = df_faixa_2010_combined['Indicador'].apply(normalizar_faixa_etaria_2010)
df_faixa_2010_combined = df_faixa_2010_combined.dropna(subset=['faixa_normalizada']) # Remove linhas que não foram mapeadas
faixa_total_2010_raw = df_faixa_2010_combined.groupby('faixa_normalizada')['Valor'].sum()
fator_2010 = total_correto_2010 / faixa_total_2010_raw.sum() if faixa_total_2010_raw.sum() > 0 else 1
faixa_total_2010 = (faixa_total_2010_raw * fator_2010).reset_index(); faixa_total_2010.columns = ["faixa_etaria", "populacao"]; faixa_total_2010["ano"] = "2010"

# --- Junção e Plot ---
faixa_df_final = pd.concat([faixa_total_2010, faixa_total_2022], ignore_index=True)
ordem_faixas = sorted(faixa_df_final['faixa_etaria'].unique(), key=lambda x: int(re.search(r'\d+', x).group(0)))
faixa_df_final['faixa_etaria_display'] = faixa_df_final['faixa_etaria'].str.replace('_', ' ').str.replace(' a ', '-').str.replace(' anos', '').str.replace(' ou mais', '+')
ordem_display = [d.replace('_', ' ').replace(' a ', '-').replace(' anos', '').replace(' ou mais', '+') for d in ordem_faixas]

if not faixa_df_final.empty:
    fig3 = px.bar(faixa_df_final, x="faixa_etaria_display", y="populacao", color="ano", barmode="group", title="População por Faixa Etária (2010 vs 2022)",
                  category_orders={"faixa_etaria_display": ordem_display},
                  labels={"faixa_etaria_display": "Faixa Etária", "populacao": "População"},
                  text_auto='.3s')
    fig3.update_xaxes(tickangle=45)
    fig3.update_traces(textangle=0, textposition="outside")
    st.plotly_chart(fig3, use_container_width=True)
else:
    st.warning("Dados de faixa etária não disponíveis para exibição.")
"""

# ==================================================
# 2. Salva o código em um arquivo .py
# ==================================================
STREAMLIT_APP_FILE = "app_dashboard.py"
with open(STREAMLIT_APP_FILE, "w", encoding="utf-8") as f:
    f.write(streamlit_code)
print(f"✅ Código do Streamlit salvo em '{STREAMLIT_APP_FILE}'")

# ==================================================
# 3. Executa o Streamlit e o ngrok
# ==================================================
def is_port_in_use(port: int) -> bool:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(("localhost", port)) == 0

# <<< IMPORTANTE: Insira seu token do ngrok abaixo >>>
# É uma boa prática não deixar tokens expostos diretamente no código,
# mas para o ambiente do Colab, esta é a forma mais direta.
NGROK_AUTH_TOKEN = "32Qi64IHMghRwMNkoFZu49G4xIF_yYjaH7aJoXcfiJEhhhCs"
if not NGROK_AUTH_TOKEN or NGROK_AUTH_TOKEN == "SEU_TOKEN_AQUI":
    print("⚠️ ATENÇÃO: Insira seu token de autenticação do ngrok para continuar.")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    PORTA_STREAMLIT = 8501
    print("🚀 Iniciando o Streamlit em background...")
    proc = subprocess.Popen(
        ["streamlit", "run", STREAMLIT_APP_FILE, "--server.port", str(PORTA_STREAMLIT), "--server.headless", "true", "--server.enableCORS", "false"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding="utf-8"
    )
    atexit.register(proc.kill)
    print("⏳ Aguardando o servidor do Streamlit ficar pronto...")
    start_time = time.time()
    while not is_port_in_use(PORTA_STREAMLIT):
        time.sleep(0.5)
        if time.time() - start_time > 60:
            print("❌ Erro: O Streamlit demorou muito para iniciar.", proc.stderr.read())
            exit()
    print("✅ Servidor Streamlit iniciado com sucesso!")
    try:
        public_url = ngrok.connect(PORTA_STREAMLIT)
        print("─" * 50); print(f"🎉 Seu dashboard está no ar!"); print(f"🔗 URL Pública: {public_url}"); print("─" * 50)
    except Exception as e:
        print(f"❌ Erro ao conectar com o ngrok: {e}")


# TENTANDO INSERIR A HOME PAGE COM O FLASK - vinícius 22/09

In [ ]:
# -*- coding: utf-8 -*-
"""
Script final que integra uma aplicação Flask com um dashboard Streamlit.

Novidades desta versão:
1.  Removido o Nginx para simplificar e corrigir erros de inicialização.
2.  Usa pyngrok para criar dois túneis públicos independentes: um para o Flask
    (o portal) e um para o Streamlit (o dashboard).
3.  O Flask injeta a URL pública do Streamlit no iframe, resolvendo o problema
    de conexão e garantindo que o dashboard sempre carregue.
4.  Mantém a tela de carregamento com CSS/JS para uma melhor experiência do usuário.
"""

# ==============================================================================
# PASSO 1: INSTALAR BIBLIOTECAS
# ==============================================================================
print("--- [1/6] Instalando bibliotecas (Flask, PyNgrok, Streamlit, Pandas, Plotly)... ---")
import os
os.system('pip install flask pyngrok streamlit pandas plotly -q')
print("Bibliotecas instaladas com sucesso!")


# ==============================================================================
# PASSO 2: IMPORTAÇÕES E CONFIGURAÇÃO
# ==============================================================================
import subprocess
import time
import atexit
import socket
from flask import Flask, render_template, url_for
from pyngrok import ngrok

print("\n--- [2/6] Criando a estrutura de pastas do projeto... ---")
os.makedirs("templates", exist_ok=True)
os.makedirs("static/css", exist_ok=True)
print("Pastas 'templates' e 'static/css' prontas.")


# ==============================================================================
# PASSO 3: CRIAR O ARQUIVO DO DASHBOARD STREAMLIT (app_dashboard.py)
# ==============================================================================
print("\n--- [3/6] Criando o arquivo do dashboard Streamlit (app_dashboard.py)... ---")

streamlit_code = r"""
import streamlit as st
import pandas as pd
import plotly.express as px
import re

st.set_page_config(
    page_title="Dashboard Populacional SJC",
    page_icon="📊",
    layout="wide"
)

# Funções auxiliares (limpeza e carregamento de dados)
@st.cache_data
def limpar_colunas(df):
    cols = df.columns.str.strip().str.lower()
    replacements = {
        ' ': '_', 'ç': 'c', 'ã': 'a', 'õ': 'o', 'á': 'a', 'é': 'e',
        'í': 'i', 'ó': 'o', 'ú': 'u', 'â': 'a', 'ê': 'e', 'î': 'i',
        'ô': 'o', 'û': 'u'
    }
    for old, new in replacements.items():
        cols = cols.str.replace(old, new, regex=False)
    cols = cols.str.replace(r'[^a-z0-9_]', '', regex=True)
    df.columns = cols
    return df

def converter_para_numerico(series):
    if pd.api.types.is_string_dtype(series):
        series = (
            series.str.replace(r"(dados alterados)", "", regex=True)
            .str.replace(r"\\.", "", regex=True)
            .str.replace(r",", ".", regex=True)
        )
    return pd.to_numeric(series, errors='coerce')

@st.cache_data
def carregar_csvs():
    urls_csv = {
        "idade_sexo_2022": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20Idade%20e%20sexo.csv",
        "pop_residencia_2022": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/RubyFox%20-%20Dados%20de%202022%20-%20popula%C3%A7%C3%A3o%20e%20residencia.csv",
        "pop_residencia_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/populacao_residente_sjc_2010.csv",
        "densidade_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/densidade_demografica_sjc_2010.csv",
        "faixa_mulheres_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/faixa_etaria_mulheres_2010.csv",
        "faixa_homens_2010": "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Arquivos%20dados%20CSV/faixa_etaria_homens_2010.csv"
    }
    try:
        datasets_raw = {key: pd.read_csv(url) for key, url in urls_csv.items()}
        datasets = {}
        for key, df in datasets_raw.items():
            if key in ["pop_residencia_2010", "faixa_homens_2010", "faixa_mulheres_2010"]:
                datasets[key] = df.copy()
            else:
                datasets[key] = limpar_colunas(df.copy())
        for col in datasets["idade_sexo_2022"].columns:
            if col not in ["tipo", "ano"]:
                datasets["idade_sexo_2022"][col] = converter_para_numerico(datasets["idade_sexo_2022"][col])
        for col in datasets["pop_residencia_2022"].columns:
            if col not in ['ano']:
                datasets["pop_residencia_2022"][col] = converter_para_numerico(datasets["pop_residencia_2022"][col])
        return datasets
    except Exception as e:
        st.error(f"Erro ao carregar dados: {e}")
        st.stop()

def normalizar_faixa_etaria_2010(indicador):
    indicador = str(indicador).lower()
    if 'menos de 1' in indicador or '1 a 4' in indicador: return '0_a_4_anos'
    if '5 a 9' in indicador: return '5_a_9_anos'
    if '10 a 14' in indicador: return '10_a_14_anos'
    if '15 a 19' in indicador: return '15_a_19_anos'
    if '20 a 24' in indicador: return '20_a_24_anos'
    if '25 a 29' in indicador: return '25_a_29_anos'
    if '30 a 34' in indicador: return '30_a_34_anos'
    if '35 a 39' in indicador: return '35_a_39_anos'
    if '40 a 44' in indicador: return '40_a_44_anos'
    if '45 a 49' in indicador: return '45_a_49_anos'
    if '50 a 54' in indicador: return '50_a_54_anos'
    if '55 a 59' in indicador: return '55_a_59_anos'
    if '60 a 64' in indicador: return '60_a_64_anos'
    if '65 a 69' in indicador: return '65_a_69_anos'
    if '70 a 74' in indicador: return '70_a_74_anos'
    if '75 a 79' in indicador: return '75_a_79_anos'
    if '80 a 84' in indicador: return '80_a_84_anos'
    if '85 a 89' in indicador: return '85_a_89_anos'
    if '90 a 94' in indicador: return '90_a_94_anos'
    if '95 a 99' in indicador: return '95_a_99_anos'
    if '100 anos ou mais' in indicador: return '100_anos_ou_mais'
    return None

# Início do Dashboard
st.title("📊 Análise Populacional - São José dos Campos")
datasets = carregar_csvs()
total_correto_2022 = datasets["pop_residencia_2022"]["populacao_residente"].iloc[0]
total_correto_2010 = datasets["pop_residencia_2010"].loc[datasets["pop_residencia_2010"]['Indicador'] == 'População residente', 'Valor'].iloc[0]

# GRÁFICO 1
st.header("População Geral")
col1, col2 = st.columns([3, 1])
with col1:
    df_idade_sexo_2022 = datasets["idade_sexo_2022"]
    homens_2022_raw = df_idade_sexo_2022[df_idade_sexo_2022["tipo"].str.contains("homens", na=False, case=False)]["total"].sum()
    mulheres_2022_raw = df_idade_sexo_2022[df_idade_sexo_2022["tipo"].str.contains("mulheres", na=False, case=False)]["total"].sum()
    soma_raw_2022 = homens_2022_raw + mulheres_2022_raw
    homens_2022 = (homens_2022_raw / soma_raw_2022) * total_correto_2022 if soma_raw_2022 > 0 else 0
    mulheres_2022 = (mulheres_2022_raw / soma_raw_2022) * total_correto_2022 if soma_raw_2022 > 0 else 0
    df_sexo_2022 = pd.DataFrame([{"Sexo": "Homens", "População": homens_2022, "Ano": "2022"}, {"Sexo": "Mulheres", "População": mulheres_2022, "Ano": "2022"}])
    homens_2010_raw = datasets["faixa_homens_2010"]['Valor'].sum()
    mulheres_2010_raw = datasets["faixa_mulheres_2010"]['Valor'].sum()
    soma_raw_2010 = homens_2010_raw + mulheres_2010_raw
    homens_2010 = (homens_2010_raw / soma_raw_2010) * total_correto_2010 if soma_raw_2010 > 0 else 0
    mulheres_2010 = (mulheres_2010_raw / soma_raw_2010) * total_correto_2010 if soma_raw_2010 > 0 else 0
    df_sexo_2010 = pd.DataFrame([{"Sexo": "Homens", "População": homens_2010, "Ano": "2010"}, {"Sexo": "Mulheres", "População": mulheres_2010, "Ano": "2010"}])
    df_sexo_final = pd.concat([df_sexo_2010, df_sexo_2022], ignore_index=True)
    fig1 = px.bar(df_sexo_final, x="Sexo", y="População", color="Ano", barmode="group", title="População por Sexo (2010 vs 2022)", text_auto='.3s')
    fig1.update_traces(textangle=0, textposition="outside")
    st.plotly_chart(fig1, use_container_width=True)
with col2:
    st.metric("População Total 2022", f"{int(total_correto_2022):,}".replace(",", "."))
    st.metric("População Total 2010", f"{int(total_correto_2010):,}".replace(",", "."))
    if total_correto_2010 > 0:
        st.metric("Crescimento Populacional", f"{(total_correto_2022/total_correto_2010 - 1):.2%}")

# GRÁFICO 2
st.header("Densidade Demográfica")
df_densidade_2010 = limpar_colunas(datasets["densidade_2010"].copy())
data_densidade = []
if "densidade_demografica_habkm2" in df_densidade_2010.columns:
    densidade_2010 = df_densidade_2010["densidade_demografica_habkm2"].iloc[0]
    data_densidade.append({"Ano": "2010", "Densidade (hab/km²)": densidade_2010})
if "densidade" in datasets["pop_residencia_2022"].columns:
    densidade_2022 = datasets["pop_residencia_2022"]["densidade"].iloc[0]
    data_densidade.append({"Ano": "2022", "Densidade (hab/km²)": densidade_2022})
if data_densidade:
    df_densidade = pd.DataFrame(data_densidade)
    fig2 = px.line(df_densidade, x="Ano", y="Densidade (hab/km²)", markers=True, title="Evolução da Densidade Demográfica")
    fig2.update_traces(text=df_densidade["Densidade (hab/km²)"].round(2), textposition="top center")
    st.plotly_chart(fig2, use_container_width=True)

# GRÁFICO 3
st.header("Distribuição da População por Faixa Etária")
df_idade_sexo_2022 = datasets["idade_sexo_2022"]
faixas_2022 = [c for c in df_idade_sexo_2022.columns if c not in ["ano", "tipo", "total"]]
faixa_total_2022_raw = df_idade_sexo_2022.loc[df_idade_sexo_2022["tipo"].str.lower().isin(["homens", "mulheres"]), faixas_2022].sum().fillna(0)
fator_2022 = total_correto_2022 / faixa_total_2022_raw.sum() if faixa_total_2022_raw.sum() > 0 else 1
faixa_total_2022 = (faixa_total_2022_raw * fator_2022).reset_index(); faixa_total_2022.columns = ["faixa_etaria", "populacao"]; faixa_total_2022["ano"] = "2022"
df_h_2010 = datasets["faixa_homens_2010"]; df_m_2010 = datasets["faixa_mulheres_2010"]
df_faixa_2010_combined = pd.concat([df_h_2010, df_m_2010])
df_faixa_2010_combined['faixa_normalizada'] = df_faixa_2010_combined['Indicador'].apply(normalizar_faixa_etaria_2010)
df_faixa_2010_combined = df_faixa_2010_combined.dropna(subset=['faixa_normalizada'])
faixa_total_2010_raw = df_faixa_2010_combined.groupby('faixa_normalizada')['Valor'].sum()
fator_2010 = total_correto_2010 / faixa_total_2010_raw.sum() if faixa_total_2010_raw.sum() > 0 else 1
faixa_total_2010 = (faixa_total_2010_raw * fator_2010).reset_index(); faixa_total_2010.columns = ["faixa_etaria", "populacao"]; faixa_total_2010["ano"] = "2010"
faixa_df_final = pd.concat([faixa_total_2010, faixa_total_2022], ignore_index=True)
ordem_faixas = sorted(faixa_df_final['faixa_etaria'].unique(), key=lambda x: int(re.search(r'\d+', x).group(0)))
faixa_df_final['faixa_etaria_display'] = faixa_df_final['faixa_etaria'].str.replace('_', ' ').str.replace(' a ', '-').str.replace(' anos', '').str.replace(' ou mais', '+')
ordem_display = [d.replace('_', ' ').replace(' a ', '-').replace(' anos', '').replace(' ou mais', '+') for d in ordem_faixas]
if not faixa_df_final.empty:
    fig3 = px.bar(faixa_df_final, x="faixa_etaria_display", y="populacao", color="ano", barmode="group", title="População por Faixa Etária (2010 vs 2022)",
                  category_orders={"faixa_etaria_display": ordem_display},
                  labels={"faixa_etaria_display": "Faixa Etária", "populacao": "População"},
                  text_auto='.3s')
    fig3.update_xaxes(tickangle=45)
    fig3.update_traces(textangle=0, textposition="outside")
    st.plotly_chart(fig3, use_container_width=True)
"""
STREAMLIT_APP_FILE = "app_dashboard.py"
with open(STREAMLIT_APP_FILE, "w", encoding="utf-8") as f:
    f.write(streamlit_code)
print(f"Arquivo '{STREAMLIT_APP_FILE}' criado com sucesso.")


# ==============================================================================
# PASSO 4: CRIAR OS ARQUIVOS DO SITE FLASK (HTML E CSS)
# ==============================================================================
print("\n--- [4/6] Criando os arquivos HTML e CSS para o portal Flask... ---")
base_url = "https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Front-End/img/"
graficos_info = {
    1: {"titulo": "População por Sexo", "imagem": f"{base_url}img_sex.jpg?raw=true"},
    2: {"titulo": "Densidade Demográfica", "imagem": f"{base_url}img_dens_demo.jpg?raw=true"},
    3: {"titulo": "População Residente", "imagem": f"{base_url}img_resid.jpg?raw=true"},
    4: {"titulo": "Faixa Etária", "imagem": f"{base_url}img_faix_etar.jpg?raw=true"},
    5: {"titulo": "Trânsito por Região", "imagem": f"{base_url}img_trans_regiao.jpg?raw=true"},
    6: {"titulo": "Serviço por Região", "imagem": f"{base_url}img_serv_regiao.jpg?raw=true"}
}
layout_html = """
<!DOCTYPE html>
<html lang="pt-br"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0"><title>{{ title }} - Prefeitura de SJC</title><link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}"></head><body><div class="bannerassistida"><nav class="menu"><a href="#">Acesso à informação</a><a href="#">Transparência SJC</a><a href="#">Legislação</a><a href="#">Ouvidoria</a></nav></div><header><img class="logo_color" src="https://github.com/FATCK06/ProjectAPI_FirstSemester/blob/main/Front-End/img/logo2.png?raw=true" alt="Logo Colorido SJC"><a href="{{ url_for('home') }}"><img class="img_logo" src="https://github.com/FATCK06/ProjectAPI_FirstSemester/blob/main/Front-End/img/logo.png?raw=true" alt="Logo da Prefeitura"></a><div><h2>Prefeitura</h2><h2>São José dos Campos</h2></div><nav><ul><li><a href="{{ url_for('home') }}"><p>Home</p></a></li><span>|</span><li><a href="#"><p>Gráficos</p></a></li><span>|</span><li><a href="#"><p>Mapa Interativo</p></a></li><span>|</span><li><a href="#"><p>Relatório</p></a></li></ul></nav></header>{% block content %}{% endblock %}<footer class="footer"><div class="footer-container"><div class="footer-logo"><img src="https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/refs/heads/main/Front-End/img/Bras%C3%A3o_de_S%C3%A3o_Jos%C3%A9_dos_Campos%201%201.svg" alt="Brasão de São José dos Campos"><div class="footer-text"><h3>Prefeitura</h3><p>São José dos Campos</p></div></div><nav class="footer-nav"><a href="{{ url_for('home') }}">Home</a><span>|</span><a href="#">Gráficos</a><span>|</span><a href="#">Mapa Interativo</a><span>|</span><a href="#">Relatório</a></nav></div></footer></body></html>
"""
with open("templates/layout.html", "w", encoding="utf-8") as f: f.write(layout_html)
index_html = """
{% extends "layout.html" %}{% block content %}<div class="banner-img"><img src="https://github.com/FATCK06/ProjectAPI_FirstSemester/blob/main/Front-End/img/20250831_181119-fotor-enhance-20250918193859.jpg?raw=true" alt="Banner da cidade de São José dos Campos"></div><main><div class="titulo-container"><h1>Resumo dos Gráficos<span class="titulo-sublinhado"></span></h1></div><div class="body_home">{% for id, info in graficos.items() %}<a href="{{ url_for('pagina_grafico', grafico_id=id) }}" class="container_home"><img src="{{ info.imagem }}" alt="Gráfico de {{ info.titulo }}"><h1>{{ info.titulo }}</h1></a>{% endfor %}</div></main>{% endblock %}
"""
with open("templates/index.html", "w", encoding="utf-8") as f: f.write(index_html)
# O IFRAME AGORA USARÁ UMA VARIÁVEL COM A URL PÚBLICA DO STREAMLIT
grafico_html = """
{% extends "layout.html" %}{% block content %}<main><div class="titulo-container"><h1>Dashboard Interativo<span class="titulo-sublinhado"></span></h1></div><div class="content-container"><p>Explore os dados demográficos de São José dos Campos nos gráficos interativos abaixo. Os dados são comparativos entre os censos de 2010 e 2022.</p><div id="loader-container" class="loader-container"><div class="loader"></div><p class="loader-text">Carregando dashboard, por favor aguarde...</p></div><iframe id="streamlit-iframe" width="100%" height="1800" style="border:none; border-radius: 8px; box-shadow: 0 4px 10px rgba(0,0,0,0.1); visibility: hidden;"></iframe></div></main><script>document.addEventListener("DOMContentLoaded", function() { const iframe = document.getElementById('streamlit-iframe'); const loader = document.getElementById('loader-container'); const streamlitUrl = "{{ streamlit_public_url }}"; iframe.onload = function() { console.log("Dashboard carregado."); loader.style.display = 'none'; iframe.style.visibility = 'visible'; }; iframe.src = streamlitUrl; });</script>{% endblock %}
"""
with open("templates/grafico.html", "w", encoding="utf-8") as f: f.write(grafico_html)
css_content = """
* { margin: 0; padding: 0; box-sizing: border-box; list-style: none; text-decoration: none; font-family: system-ui, -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif; }
body { background-color: #fdfdfd; }
.bannerassistida { width: 100%; height: 50px; background-color: #EC6608; display: flex; align-items: center; padding-left: 20px; }
.menu a { color: white; margin: 0 15px; text-decoration: none; font-weight: bold; font-size: 14px; }
header { display: flex; align-items: center; gap: 15px; padding: 20px 30px; border-bottom: 4px solid #FFBF00; background-color: white; }
header div h2 { margin: 0; line-height: 1.1; }
.img_logo { width: 60px; height: auto; } .logo_color { width: 40px; height: auto; }
header nav { margin-left: auto; }
header nav ul { display: flex; align-items: center; gap: 15px; }
header nav ul li a { color: black; font-size: 16px; transition: color 0.3s ease; }
header nav ul li a:hover { color: #EC6608; }
header nav ul span { color: #ccc; }
.banner-img { width: 100%; max-height: 450px; overflow: hidden; background-color: #eee; }
.banner-img img { width: 100%; height: 100%; object-fit: cover; display: block; }
main { padding: 20px; min-height: 60vh; }
.titulo-container { text-align: center; margin: 30px 0 40px 0; }
.titulo-container h1 { display: inline-block; position: relative; padding-bottom: 10px; font-size: 2.2rem; color: #333; }
.titulo-sublinhado { position: absolute; left: 50%; transform: translateX(-50%); bottom: 0; width: 120%; height: 4px; background-color: #EC6608; display: block; }
.body_home { display: grid; grid-template-columns: repeat(auto-fit, minmax(350px, 1fr)); gap: 30px; max-width: 1200px; margin: 30px auto; }
.container_home { position: relative; overflow: hidden; border-radius: 10px; box-shadow: 0 4px 10px rgba(0,0,0,0.1); display: block; color: white; }
.container_home img { width: 100%; height: 100%; object-fit: cover; display: block; border-radius: 10px; filter: grayscale(100%); transition: filter 0.4s ease, transform 0.4s ease; }
.container_home:hover img { filter: grayscale(0%); transform: scale(1.05); }
.container_home h1 { position: absolute; bottom: 15px; left: 20px; font-size: 2rem; font-weight: bold; text-shadow: 2px 2px 8px rgba(0,0,0,0.9); }
.content-container { max-width: 95%; margin: 0 auto; padding: 10px; background-color: #fff; border-radius: 8px; }
.content-container p { font-size: 1.1rem; line-height: 1.6; margin-bottom: 20px; color: #555; text-align: center; }
.loader-container { display: flex; justify-content: center; align-items: center; min-height: 600px; flex-direction: column; gap: 20px; }
.loader { border: 8px solid #f3f3f3; border-top: 8px solid #0d57a5; border-radius: 50%; width: 60px; height: 60px; animation: spin 1.5s linear infinite; }
.loader-text { font-size: 1.2rem; color: #555; font-weight: 500; }
@keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }
.footer { background-color: #0d57a5; color: white; padding: 25px 40px; margin-top: 50px; }
.footer-container { display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap; gap: 20px; max-width: 1200px; margin: 0 auto; }
.footer-logo { display: flex; align-items: center; gap: 15px; }
.footer-logo img { width: 60px; height: 60px; border-radius: 50%; background-color: #fff; }
.footer-text h3 { margin: 0; font-size: 14px; font-weight: bold; }
.footer-text p { margin: 0; font-size: 12px; }
.footer-nav { display: flex; align-items: center; gap: 15px; }
.footer-nav a { color: white; font-size: 14px; transition: color 0.3s ease; }
.footer-nav a:hover { color: #FFBF00; }
.footer-nav span { color: rgba(255, 255, 255, 0.6); }
@media (max-width: 768px) {
    header { flex-direction: column; gap: 20px; }
    header nav { margin-left: 0; }
    .titulo-container h1 { font-size: 1.8rem; }
    .container_home h1 { font-size: 1.5rem; }
    .footer-container { flex-direction: column; justify-content: center; text-align: center; }
}
"""
with open("static/css/style.css", "w", encoding="utf-8") as f: f.write(css_content)
print("Arquivos do portal Flask criados com sucesso.")


# ==============================================================================
# PASSO 5: INICIAR SERVIDORES (STREAMLIT E FLASK)
# ==============================================================================
print("\n--- [5/6] Configurando e iniciando os servidores... ---")

# --- Função para verificar se uma porta está em uso ---
def is_port_in_use(port: int) -> bool:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(("localhost", port)) == 0

# --- Configurações ---
NGROK_AUTH_TOKEN = "32Qi64IHMghRwMNkoFZu49G4xIF_yYjaH7aJoXcfiJEhhhCs" # <<< SUBSTITUA PELO SEU TOKEN
FLASK_PORT = 5000
STREAMLIT_PORT = 8501
streamlit_public_url = None # Variável global para a URL do Streamlit

# --- Configurar token do Ngrok ---
if not NGROK_AUTH_TOKEN or "COLE_SEU_TOKEN_AQUI" in NGROK_AUTH_TOKEN:
    print("⚠️  ATENÇÃO: Token do Ngrok não configurado. Sessão será limitada.")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ Token de autenticação do Ngrok configurado.")

# --- Iniciar Streamlit em segundo plano ---
print(f"🚀 Iniciando o Streamlit em background na porta {STREAMLIT_PORT}...")
proc = subprocess.Popen(
    ["streamlit", "run", STREAMLIT_APP_FILE, "--server.port", str(STREAMLIT_PORT), "--server.headless", "true"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding="utf-8"
)
atexit.register(proc.kill)

# --- Aguardar o Streamlit iniciar ---
print("⏳ Aguardando o servidor do Streamlit ficar pronto...")
start_time = time.time()
while not is_port_in_use(STREAMLIT_PORT):
    time.sleep(0.5)
    if time.time() - start_time > 90:
        print("❌ ERRO: Streamlit demorou muito para iniciar.", proc.stderr.read())
        exit()
print("✅ Servidor Streamlit iniciado.")


# ==============================================================================
# PASSO 6: CONFIGURAR FLASK E INICIAR TÚNEIS NGROK
# ==============================================================================
app = Flask(__name__)

@app.route("/")
def home():
    return render_template('index.html', title="Home", graficos=graficos_info)

@app.route("/grafico/<int:grafico_id>")
def pagina_grafico(grafico_id):
    global streamlit_public_url
    grafico = graficos_info.get(grafico_id)
    if not grafico:
        return "Gráfico não encontrado!", 404
    # Passa a URL pública do Streamlit para o template do gráfico
    return render_template('grafico.html', title=grafico["titulo"], streamlit_public_url=streamlit_public_url)

# --- Inicia os túneis e a aplicação ---
try:
    ngrok.kill()
    # Cria o túnel para o Streamlit e armazena sua URL pública
    streamlit_tunnel = ngrok.connect(STREAMLIT_PORT, name="streamlit")
    streamlit_public_url = streamlit_tunnel.public_url
    print(f"✅ Túnel do Streamlit está pronto em: {streamlit_public_url}")

    # Cria o túnel para o Flask
    flask_tunnel = ngrok.connect(FLASK_PORT, name="flask")
    print("\n" + "─" * 60)
    print(f"🎉 APLICAÇÃO PRINCIPAL (PORTAL) ESTÁ NO AR! 🎉")
    print(f"🔗 ACESSE POR ESTA URL: {flask_tunnel.public_url}")
    print("─" * 60)

    # Executa a aplicação Flask (bloqueia a execução aqui, mantendo tudo no ar)
    app.run(port=FLASK_PORT)

except Exception as e:
    print(f"❌ Erro ao iniciar Ngrok ou Flask: {e}")



# Geração do mapa interativo da cidade de São José dos Campos:

Aqui é onde o mapa interativo de SJC será inserida no Google Colab. Logo logo será inserido na home page

Geração do mapa interativo - Versão Final

In [ ]:
import folium
import re
import json
import geopandas as gpd
from shapely.geometry import mapping
from branca.element import Element
import unicodedata
import difflib

# ---------- Config ----------
url_geojson = 'https://raw.githubusercontent.com/FATCK06/ProjectAPI_FirstSemester/main/Delimita%C3%A7%C3%A3o%20da%20Zona%20de%20SJC/zonas_sjc_poligono.geojson'

# Seus dados (população e área aproximada — usados para população/densidade)
zonas = {
    "Zona Norte": {'populacao': 100000, 'area': 23.35},
    "Zona Sul": {'populacao': 90000, 'area': 32.69},
    "Zona Leste": {'populacao': 120000, 'area': 28.02},
    "Zona Centro": {'populacao': 72401, 'area': 18.68},
    "Zona Oeste": {'populacao': 64482, 'area': 25.66},
    "Zona Sudeste": {'populacao': 62541, 'area': 30.37}
}

cores = {
    "Zona Norte": "#e41a1c",
    "Zona Sul": "#377eb8",
    "Zona Leste": "#4daf4a",
    "Zona Oeste": "#984ea3",
    "Zona Centro": "#ff7f00",
    "Zona Sudeste": "#ffff33"
}

# ---------- Carregar GeoJSON ----------
gdf = gpd.read_file(url_geojson)

# Garante que o GeoDataFrame tem CRS; assumimos EPSG:4326 se estiver ausente
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)

# Reprojetar para UTM (métrico) para cálculo preciso de áreas/centroides
gdf_proj = gdf.to_crs(epsg=32723)  # UTM zone 23S

# Calcula área em km² a partir da geometria reprojetada
gdf['area_km2'] = gdf_proj.geometry.area / 1e6  # em km² (float)

# Calcula centroides em coordenadas WGS84 (lat/lon) para posicionar marcadores
centroids_proj = gdf_proj.centroid  # em UTM
centroids_wgs = centroids_proj.to_crs(epsg=4326)
gdf['centroid_lat'] = centroids_wgs.y
gdf['centroid_lon'] = centroids_wgs.x

# Preparar mapeamento de nomes com normalização para fuzzy match
def norm(s):
    if s is None:
        return ""
    s = str(s)
    # remove acentos com unicodedata, converte para ascii
    s = unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('ascii')
    s = s.lower()
    s = re.sub(r'[^a-z0-9]+', ' ', s).strip()
    return s

zonas_norm_map = { norm(k): k for k in zonas.keys() }  # norm_name -> original_key

# ---------- Criar mapa ----------
map_center = [gdf['centroid_lat'].mean(), gdf['centroid_lon'].mean()]
mapa = folium.Map(location=map_center, zoom_start=12, control_scale=True)

# Guardar funções JS para alternar polígonos
js_functions = []

# Iterar features e criar markers/popups + JS toggle
for idx, row in gdf.iterrows():
    nome_raw = row.get('nome', '')  # nome vindo do GeoJSON
    nome_norm = norm(nome_raw)

    # fuzzy match para encontrar a chave mais próxima em zonas, se existir
    possible = difflib.get_close_matches(nome_norm, list(zonas_norm_map.keys()), n=1, cutoff=0.6)
    matched_key = zonas_norm_map[possible[0]] if possible else None

    populacao = zonas[matched_key]['populacao'] if matched_key else None
    # área a exibir: calculada a partir da geometria (mais precisa)
    area_calc = float(row['area_km2'])
    densidade = (populacao / area_calc) if (populacao and area_calc>0) else None
    cor = cores.get(matched_key if matched_key else nome_raw, "#3388ff")

    # centroid for marker
    lat_c = float(row['centroid_lat'])
    lon_c = float(row['centroid_lon'])

    # safe JS id
    safe = re.sub(r'\W+', '_', nome_raw if nome_raw else f'zona_{idx}')

    # converter geometria para GeoJSON literal (objeto JS)
    geom_json = json.dumps(mapping(row.geometry))

    # JS function (toggle) — usa L.geoJSON com o objeto GeoJSON
    js_fn = f"""
function togglePoly_{safe}(){{
  var map = {mapa.get_name()};
  if(window['poly_{safe}']){{
    map.removeLayer(window['poly_{safe}']);
    window['poly_{safe}'] = null;
  }} else {{
    window['poly_{safe}'] = L.geoJSON({geom_json}, {{
      style: function (feature) {{
        return {{color: '{cor}', weight: 2, fillOpacity: 0.45}};
      }}
    }}).addTo(map);
    try {{ map.fitBounds(window['poly_{safe}'].getBounds()); }} catch(e){{}}
  }}
}}
"""
    js_functions.append(js_fn)

    # montar popup com valores formatados e fallback '---'
    pop_str = f"{int(populacao):,}".replace(',', '.') if populacao else "---"
    area_str = f"{area_calc:.2f}"
    dens_str = f"{densidade:.2f} hab/km²" if densidade else "---"

    popup_html = f"""
    <div style="font-family: Arial, sans-serif; font-size:13px;">
      <b>{nome_raw if nome_raw else 'Sem nome'}</b><br>
      População: {pop_str}<br>
      Área: {area_str} km²<br>
      Densidade: {dens_str}<br>
      <a href="#" onclick="togglePoly_{safe}();return false;">Mostrar/Esconder área</a>
    </div>
    """

    folium.Marker(
        location=[lat_c, lon_c],
        popup=folium.Popup(popup_html, max_width=330),
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(mapa)

# injeta o JS no HTML do mapa
script_all = "<script>\n" + "\n".join(js_functions) + "\n</script>"
mapa.get_root().html.add_child(Element(script_all))

# Exibir mapa
mapa
